In [34]:
import ast
import glob
import gzip
import json
import os
import pickle
import re
import numpy as np
import pandas as pd

## Extracción de datos

In [5]:
# Almacenaremos la data extraída en un diccionario
datasets_raw: dict[pd.DataFrame] = {}

# Extraemos los datasets (archivos que terminan en '.json.gz')
for filepath in glob.glob('*.json.gz'):
    with gzip.open(filepath, 'rb') as f:
        try:
            # Desanidamos el archivo extraído JSON 
            data = [json.loads(line.decode('utf-8')) for line in f]
        except json.JSONDecodeError as e:
            # En caso que no se pueda desanidar con json.loads(), utilizamos ast.literal_eval() 
            data = [ast.literal_eval(line.decode('utf-8')) for line in f]
    
    # Obtenemos el nombre del archivo dentro del string de la ruta
    fname = filepath.split('/')[-1].split('.')[0]  
    # Guardamos la data como un dataframe dentro de 'datasets_raw'
    datasets_raw[fname] = pd.DataFrame(data)

In [5]:
datasets_raw.keys()

dict_keys(['users_items', 'user_reviews', 'steam_games'])

In [4]:
# Asignamos los dataframes en 'datasets_raw' a variables
df_steamgames_raw: pd.DataFrame = datasets_raw['steam_games']
df_userreviews_raw: pd.DataFrame = datasets_raw['user_reviews']
df_usersitems_raw: pd.DataFrame = datasets_raw['users_items']

## Limpieza y Transformación de Datasets

### Limpieza de nulos general
* Verificamos si existen filas con todo nulos.

In [6]:
# Existen filas con todo nulos para 'steam_games'?
df_steamgames_raw.isna().all(axis=1).any()

True

In [6]:
# Existen filas con todo nulos para 'user_reviews'?
df_userreviews_raw.isna().all(axis=1).any()

False

In [7]:
# Existen filas con todo nulos para 'users_items'?
df_usersitems_raw.isna().all(axis=1).any()

False

* Existen filas con todos nulos en 'steam_games', por ende serán eliminadas.

In [7]:
# Hacemos drop a las filas con todos nulos de 'steam_games'
df_steamgames_raw.dropna(axis=0, how='all', inplace=True)

# Confirmamos el drop
df_steamgames_raw.isna().all(axis=1).any()

False

### Dataset: steam_games

In [8]:
# Inspeccionamos el dataframe de 'steam_games' y su contenido
df_steamgames_raw.info()
df_steamgames_raw.head()

<class 'pandas.core.frame.DataFrame'>
Index: 32135 entries, 88310 to 120444
Data columns (total 13 columns):
 #   Column        Non-Null Count  Dtype 
---  ------        --------------  ----- 
 0   publisher     24083 non-null  object
 1   genres        28852 non-null  object
 2   app_name      32133 non-null  object
 3   title         30085 non-null  object
 4   url           32135 non-null  object
 5   release_date  30068 non-null  object
 6   tags          31972 non-null  object
 7   reviews_url   32133 non-null  object
 8   specs         31465 non-null  object
 9   price         30758 non-null  object
 10  early_access  32135 non-null  object
 11  id            32133 non-null  object
 12  developer     28836 non-null  object
dtypes: object(13)
memory usage: 3.4+ MB


,publisher,genres,app_name,title,url,release_date,tags,reviews_url,specs,price,early_access,id,developer
88310,Kotoshiro,"[Action, Casual, Indie, Simulation, Strategy]",Lost Summoner Kitty,Lost Summoner Kitty,http://store.steampowered.com/app/761140/Lost_...,2018-01-04,"[Strategy, Action, Indie, Casual, Simulation]",http://steamcommunity.com/app/761140/reviews/?...,[Single-player],4.99,False,761140,Kotoshiro
88311,"Making Fun, Inc.","[Free to Play, Indie, RPG, Strategy]",Ironbound,Ironbound,http://store.steampowered.com/app/643980/Ironb...,2018-01-04,"[Free to Play, Strategy, Indie, RPG, Card Game...",http://steamcommunity.com/app/643980/reviews/?...,"[Single-player, Multi-player, Online Multi-Pla...",Free To Play,False,643980,Secret Level SRL
88312,Poolians.com,"[Casual, Free to Play, Indie, Simulation, Sports]",Real Pool 3D - Poolians,Real Pool 3D - Poolians,http://store.steampowered.com/app/670290/Real_...,2017-07-24,"[Free to Play, Simulation, Sports, Casual, Ind...",http://steamcommunity.com/app/670290/reviews/?...,"[Single-player, Multi-player, Online Multi-Pla...",Free to Play,False,670290,Poolians.com
88313,彼岸领域,"[Action, Adventure, Casual]",弹炸人2222,弹炸人2222,http://store.steampowered.com/app/767400/2222/,2017-12-07,"[Action, Adventure, Casual]",http://steamcommunity.com/app/767400/reviews/?...,[Single-player],0.99,False,767400,彼岸领域
88314,NaN,NaN,Log Challenge,NaN,http://store.steampowered.com/app/773570/Log_C...,NaN,"[Action, Indie, Casual, Sports]",http://steamcommunity.com/app/773570/reviews/?...,"[Single-player, Full controller support, HTC V...",2.99,False,773570,NaN


In [9]:
# Revisamos la cantidad de nulos por columna
df_steamgames_raw.isna().sum()

publisher       8052
genres          3283
app_name           2
title           2050
url                0
release_date    2067
tags             163
reviews_url        2
specs            670
price           1377
early_access       0
id                 2
developer       3299
dtype: int64

#### Eliminación de filas nulas (steam_games)

* La información en la columnas 'genres', 'release_date', 'tags', 'specs' y 'id' son esenciales para este proyecto. Eliminamos filas con nulos en estas columnas.

In [10]:
# Eliminamos las filas con nulos en 'genres', 'release_date', 'tags', 'specs' y/o 'id'
df_steamgames_raw.dropna(subset=['genres', 'release_date', 'tags', 'specs', 'id'], inplace=True)

# Volvemos a revisar los nulos
df_steamgames_raw.isna().sum()

publisher       4892
genres             0
app_name           1
title              1
url                0
release_date       0
tags               0
reviews_url        0
specs              0
price           1199
early_access       0
id                 0
developer        170
dtype: int64

#### Eliminación de filas no-útil (steam_games)

* Identificamos filas con insuficiente información para ser útil, y las eliminamos.

In [11]:
# Primero creamos una column con un conteo de los nulos por fila
df_steamgames_raw['cantidad_nan'] = df_steamgames_raw.isna().sum(axis=1)

# Visualizamos las ocurrencias de nan y las suma de filas respectivas
df_steamgames_raw.groupby('cantidad_nan').size()

cantidad_nan
0    22530
1     5742
2      240
3       12
5        1
dtype: int64

* Visualizando 'cantidad_nan' y el CSV del dataset nos fijamos que las filas con 3, o más, nulos no son útil para el enfoque de este proyecto.

In [12]:
# Mantenemos filas con 2 nulos, o menos. Eliminamos el resto.
max_nulos_aceptables = 2
df_steamgames_raw.dropna(thresh=df_steamgames_raw.shape[1]-max_nulos_aceptables, axis=0, inplace=True)

# Visualizamos las ocurrencias de nan de nuevo para confirmar el dropna()
df_steamgames_raw.groupby('cantidad_nan').size()

cantidad_nan
0    22530
1     5742
2      240
dtype: int64

In [13]:
# Finalmente, eliminamos 'cantidad_nan'.
df_steamgames_raw.drop('cantidad_nan', axis=1, inplace=True)

#### Transformación de columna 'release_date' a 'release_year' (steam_games)
* En cuanto a fechas, requerimos solo información de años. Podemos optimizar nuestro dataset extrayendo los años de la columna 'release_date'.

In [14]:
# Creamos una función que nos permita extraer los 4 dígitos del year/año
def extraer_year(texto: str):
    """Encuentra un año de cuatro dígitos en el texto proporcionada.
    
    Parámetros:
    texto (str): El texto en la que se buscará el año.
    Devuelve:
    str o None: Un string que contiene el año si se encuentra, o 'None' si no se encuentra.
    """
    # Buscamos números de 4 dígitos en el string y lo retornamos
    year: str = re.search(r'\d{4}', texto)
    return year.group() if year else None

In [15]:
# Primero convertimos la columna 'release_date' a tipo string para poder aplicar la función extraer_year()
df_steamgames_raw['release_date'] = df_steamgames_raw['release_date'].astype(str)

# Aplicamos extraer_year() a 'release_date' y almacenamos el retorno en la columna 'release_year'
df_steamgames_raw['release_year'] = df_steamgames_raw['release_date'].apply(extraer_year)

# Convertimos columna 'release_year' a tipo datetime
df_steamgames_raw['release_year'] = pd.to_datetime(df_steamgames_raw['release_year'], format='%Y', errors='coerce')

# Eliminamos 'release_date' que queda redundante
df_steamgames_raw.drop('release_date', axis=1, inplace=True)

In [16]:
# Si existen nulos resultantes de la creación de 'release_year', los eliminamos.
df_steamgames_raw.dropna(subset='release_year', inplace=True)

#### Extracción de listas anidadas (steam_games)

In [17]:
# Creamos una función que desanida columnas especificas
# y devuelve un nuevo dataframe de valores tipo bool
def desanidar_lista_a_df(df: pd.DataFrame, col_anidada: str, col_indice: str) -> pd.DataFrame:
    """Retorna un nuevo dataframe, con 'col_indice' como indice y todo valor único en 
    'col_anidada' como columnas, de datos anidados en 'df'.
    
    Parámetros:
    df (pd.DataFrame): DataFrame que contiene una columna con datos anidados.
    col_anidada (str): El nombre de la columna con datos anidados.
    col_indice (str): El nombre de la columna que se convertirá en el indices del nuevo dataframe.
    Devuelve:
    pd.DataFrame: Un dataframe con los datos desanidados.
    """

    # Extraemos los valores únicos de 'col_anidada'
    valores_unicos = set(val for lista in df[col_anidada] for val in lista)

    # Creamos un dataframe, con 'col_indice' como indice
    df_desanidado_nuevo = pd.DataFrame(index=df[col_indice])
    
    # Agregamos columnas por cada valor único en 'col_anidada'
    for val in valores_unicos:
        df_desanidado_nuevo[val] = False

    # Iteramos por las filas y asignamos el valor True donde aplica
    for _, fila in df.iterrows():
        for val in fila[col_anidada]:
            df_desanidado_nuevo.at[fila[col_indice], val] = True
    
    return df_desanidado_nuevo

In [ ]:
# Desanidamos las columnas 'genres', 'tags' y 'specs', y los guardamos como dataframes nuevos. 
df_steamgames_genres = desanidar_lista_a_df(df_steamgames_raw, 'genres', 'id')
df_steamgames_tags = desanidar_lista_a_df(df_steamgames_raw, 'tags', 'id')
df_steamgames_specs = desanidar_lista_a_df(df_steamgames_raw, 'specs', 'id')

* Con los datos extraídos de 'genres', 'tags' y 'specs', tenemos que asegurar que no exite información redundante.

In [28]:
# Columnas comunes entre 'genres' y 'tags'
cols_comun_genres_tags = df_steamgames_genres.columns.intersection(df_steamgames_tags.columns)
cols_comun_genres_tags

Index(['Strategy', 'Casual', 'RPG', 'Utilities', 'Free to Play', 'Education',
       'Indie', 'Sports', 'Software Training', 'Photo Editing',
       'Web Publishing', 'Adventure', 'Action', 'Simulation', 'Early Access',
       'Racing', 'Audio Production', 'Video Production',
       'Massively Multiplayer'],
      dtype='object')

In [29]:
# Columnas comunes entre 'tags' y 'specs'
cols_comun_tags_specs = df_steamgames_tags.columns.intersection(df_steamgames_specs.columns)
cols_comun_tags_specs

Index(['Co-op'], dtype='object')

In [30]:
# Eliminamos las columnas comunes redundantes
df_steamgames_tags.drop(cols_comun_genres_tags, axis=1, inplace=True)
df_steamgames_specs.drop(cols_comun_tags_specs, axis=1, inplace=True)

In [18]:
# Finalmente descartamos las columnas que desanidamos y guardamos el dataframe procesado
df_steamgames_clean = df_steamgames_raw.drop(['genres', 'tags', 'specs'], axis=1)

### Dataset: user_reviews

In [21]:
# Inspeccionamos el dataframe de 'user_reviews' y su contenido
df_userreviews_raw.info()
df_userreviews_raw.head()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 25798 entries, 0 to 25797
Data columns (total 3 columns):
 #   Column    Non-Null Count  Dtype 
---  ------    --------------  ----- 
 0   user_id   25798 non-null  object
 1   user_url  25798 non-null  object
 2   reviews   25798 non-null  object
dtypes: object(3)
memory usage: 604.8+ KB


,user_id,user_url,reviews
0,js41637,http://steamcommunity.com/id/js41637,"[{'funny': '', 'posted': 'Posted June 24, 2014..."
1,evcentric,http://steamcommunity.com/id/evcentric,"[{'funny': '', 'posted': 'Posted February 3.',..."
2,doctr,http://steamcommunity.com/id/doctr,"[{'funny': '', 'posted': 'Posted October 14, 2..."
3,maplemage,http://steamcommunity.com/id/maplemage,"[{'funny': '3 people found this review funny',..."
4,Wackky,http://steamcommunity.com/id/Wackky,"[{'funny': '', 'posted': 'Posted May 5, 2014.'..."


* Observamos que la columna 'reviews' tiene datos anidados. Lo tenemos que extraer, pero primero revisamos si existen datos nulos.

In [22]:
# Revisamos la cantidad de nulos por columna
df_userreviews_raw.isna().sum()

user_id     0
user_url    0
reviews     0
dtype: int64

* No existen nulos. Procedemos a desanidar 'reviews'.

#### Extracción de datos anidadas en 'reviews' (user_reviews)

In [23]:
# Desanidamos la columna 'reviews'
df_userreviews_desanidado = df_userreviews_raw.explode('reviews', ignore_index=False)
df_userreviews_desanidado = df_userreviews_desanidado['reviews'].apply(pd.Series).join(df_userreviews_raw['user_id'])
df_userreviews_desanidado.head()

,funny,posted,last_edited,item_id,helpful,recommend,review,0,user_id
0,,"Posted June 24, 2014.",,251610,15 of 20 people (75%) found this review helpful,True,I know what you think when you see this title ...,NaN,js41637
0,,"Posted September 8, 2013.",,227300,0 of 1 people (0%) found this review helpful,True,For a simple (it's actually not all that simpl...,NaN,js41637
0,,"Posted November 29, 2013.",,239030,1 of 4 people (25%) found this review helpful,True,Very fun little game to play when your bored o...,NaN,js41637
1,,Posted February 3.,,248820,No ratings yet,True,A suitably punishing roguelike platformer. Wi...,NaN,evcentric
1,,"Posted December 4, 2015.","Last edited December 5, 2015.",370360,No ratings yet,True,"""Run for fun? What the hell kind of fun is that?""",NaN,evcentric


In [24]:
# Eliminamos una columna nula que resultó del desanidado
df_userreviews_desanidado.drop(0, axis=1, inplace=True)

#### Transformación de columna 'posted' y 'last_edited' a tipo datetime (user_reviews)

In [25]:
# Convertimos 'posted' y 'last_edited' a datos tipo datetime
df_userreviews_desanidado['posted'] = pd.to_datetime(df_userreviews_desanidado['posted'], format='Posted %B %d, %Y.', errors='coerce')
df_userreviews_desanidado['last_edited'] = pd.to_datetime(df_userreviews_desanidado['last_edited'], format='Last edited %B %d, %Y.', errors='coerce')
df_userreviews_desanidado.head()

,funny,posted,last_edited,item_id,helpful,recommend,review,user_id
0,,2014-06-24,NaT,251610,15 of 20 people (75%) found this review helpful,True,I know what you think when you see this title ...,js41637
0,,2013-09-08,NaT,227300,0 of 1 people (0%) found this review helpful,True,For a simple (it's actually not all that simpl...,js41637
0,,2013-11-29,NaT,239030,1 of 4 people (25%) found this review helpful,True,Very fun little game to play when your bored o...,js41637
1,,NaT,NaT,248820,No ratings yet,True,A suitably punishing roguelike platformer. Wi...,evcentric
1,,2015-12-04,2015-12-05,370360,No ratings yet,True,"""Run for fun? What the hell kind of fun is that?""",evcentric


#### Revisión de nulos (user_reviews)

In [26]:
# Volvemos a ver la cantidad de nulos por columna
# después del desanidado
df_userreviews_desanidado.isna().sum()

funny             28
posted         10147
last_edited    55053
item_id           28
helpful           28
recommend         28
review            28
user_id            0
dtype: int64

* Observamos 28 filas nulas nuevas. Las eliminamos, utilizando la columna 'review' como referencia

In [28]:
# Eliminamos los nulos nuevos y guardamos el dataframe procesado
df_userreviews_clean = df_userreviews_desanidado.dropna(subset='review')

### Dataset: users_items

In [31]:
# Inspeccionamos el dataframe de 'users_items' y su contenido
df_usersitems_raw.info()
df_usersitems_raw.head()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 88309 entries, 0 to 88308
Data columns (total 5 columns):
 #   Column       Non-Null Count  Dtype 
---  ------       --------------  ----- 
 0   user_id      88309 non-null  object
 1   items_count  88309 non-null  int64 
 2   steam_id     88309 non-null  object
 3   user_url     88309 non-null  object
 4   items        88309 non-null  object
dtypes: int64(1), object(4)
memory usage: 3.4+ MB


,user_id,items_count,steam_id,user_url,items
0,js41637,888,76561198035864385,http://steamcommunity.com/id/js41637,"[{'item_id': '10', 'item_name': 'Counter-Strik..."
1,evcentric,137,76561198007712555,http://steamcommunity.com/id/evcentric,"[{'item_id': '1200', 'item_name': 'Red Orchest..."
2,Riot-Punch,328,76561197963445855,http://steamcommunity.com/id/Riot-Punch,"[{'item_id': '10', 'item_name': 'Counter-Strik..."
3,doctr,541,76561198002099482,http://steamcommunity.com/id/doctr,"[{'item_id': '300', 'item_name': 'Day of Defea..."
4,MinxIsBetterThanPotatoes,371,76561198004744620,http://steamcommunity.com/id/MinxIsBetterThanP...,"[{'item_id': '50', 'item_name': 'Half-Life: Op..."


* Observamos que la columna 'items' tiene datos anidados. Lo tenemos que extraer, pero primero revisamos si existen datos nulos.

In [32]:
# Revisamos la cantidad de nulos por columna
df_usersitems_raw.isna().sum()

user_id        0
items_count    0
steam_id       0
user_url       0
items          0
dtype: int64

* No existen nulos.

#### Extracción de datos anidadas en 'items' (users_items)

In [ ]:
# Por cuestión de limite de memoria, tenemos que dividir el dataframe y procesarlo en porciones
df_usersitems_split = np.array_split(df_usersitems_raw, 100)

# Desanidamos en pasos, guardando los df procesados en una lista
dfs_usersitems_explode = []
for df in df_usersitems_split:
    df_explode = df.explode('items')
    df_explode = df_explode['items'].apply(pd.Series).join(df)
    dfs_usersitems_explode.append(df_explode)

# Consolidamos los dataframes procesados
df_usersitems_desanidado = pd.concat(dfs_usersitems_explode, axis=0)

In [34]:
# Eliminamos la columna que desanidamos en el paso anterior
# y confirmamos la transformación.
df_usersitems_desanidado.drop(['items', 0], axis=1, inplace=True)
df_usersitems_desanidado.head()

,item_id,item_name,playtime_forever,playtime_2weeks,user_id,items_count,steam_id,user_url
0,10,Counter-Strike,0.0,0.0,js41637,888,76561198035864385,http://steamcommunity.com/id/js41637
0,80,Counter-Strike: Condition Zero,0.0,0.0,js41637,888,76561198035864385,http://steamcommunity.com/id/js41637
0,100,Counter-Strike: Condition Zero Deleted Scenes,0.0,0.0,js41637,888,76561198035864385,http://steamcommunity.com/id/js41637
0,300,Day of Defeat: Source,220.0,0.0,js41637,888,76561198035864385,http://steamcommunity.com/id/js41637
0,30,Day of Defeat,0.0,0.0,js41637,888,76561198035864385,http://steamcommunity.com/id/js41637


In [35]:
# Revisamos la cantidad de nulos por columna
df_usersitems_desanidado.isna().sum()

item_id             16806
item_name           16806
playtime_forever    16806
playtime_2weeks     16806
user_id                 0
items_count             0
steam_id                0
user_url                0
dtype: int64

* Observamos 16806 filas nulas nuevas. Las eliminamos, utilizando la columna 'item_id' como referencia, y guardamos el dataframe procesado.

In [38]:
df_usersitems_clean = df_usersitems_desanidado.dropna(subset=['item_id'])

## Almacenamiento de Datos Procesados

In [42]:
# Almacenamos los dataframes en disco para uso en el proceso EDA, cargados en 'EDA.ipynb'

nombre_dir = 'data_pickle'
protocol_pkl = pickle.HIGHEST_PROTOCOL

# Creamos el directorio de almacén, si no existe
if not os.path.exists(nombre_dir):
    os.makedirs(nombre_dir)

# Datasets de steam_games a pickle
df_steamgames_clean.to_pickle(f'./{nombre_dir}/df_steamgames_clean.pkl', protocol=protocol_pkl)
df_steamgames_genres.to_pickle(f'./{nombre_dir}/df_steamgames_genres.pkl', protocol=protocol_pkl)
df_steamgames_tags.to_pickle(f'./{nombre_dir}/df_steamgames_tags.pkl', protocol=protocol_pkl)
df_steamgames_specs.to_pickle(f'./{nombre_dir}/df_steamgames_specs.pkl', protocol=protocol_pkl)

# Dataset de user_reviews a pickle
df_userreviews_clean.to_pickle(f'./{nombre_dir}/df_userreviews_clean.pkl', protocol=protocol_pkl)

# Dataset de users_items a pickle
df_usersitems_clean.to_pickle(f'./{nombre_dir}/df_usersitems_clean.pkl', protocol=protocol_pkl)